In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
from datetime import datetime, date, timedelta, timezone
import pytz as tz
from pytz import timezone

In [2]:
def obtener_stats(func_stats):
    stats_dict = {}
    
    date = func_stats.find('div', attrs={'class':'info match-link'})
    if date:
        stats_dict['date'] = date.get('starttime').replace('T',' ').replace(':00+00:00','')
    else:
        stats_dict['date'] = '-'
 
    zerozero = func_stats.find('div', attrs={'class':'content-box draw'}).find('div', attrs={'class':'cont'})
    if zerozero:
        stats_dict['zerozero'] = zerozero.get_text('/',strip=True).replace('%','')[4:10].replace('<','')
    else:
        stats_dict['zerozero'] = '-' 
        
    homeRank = func_stats.find('div', attrs={'class':'position t1'})
    if homeRank:
        stats_dict['homeRank'] = homeRank.get_text()
    else:
        stats_dict['homeRank'] = '-'    
            
    homeTeam = func_stats.find('p', attrs={'itemprop':'homeTeam'})
    if homeTeam:
        stats_dict['homeTeam'] = homeTeam.get_text('/', strip=True)
    else:
        stats_dict['homeTeam'] = '-'

    homeVictory = func_stats.find('div', attrs={'class':'content-box t-1'}).find('div', attrs={'class':'poss-box'})
    if homeVictory:
        stats_dict['homeVictory'] = homeVictory.get_text('/', strip=True).replace('/Probabilidad gana/',' - ')[0:2].replace('.','').replace('%','')
    else:
        stats_dict['homeVictory'] = '-'     
        
    homeGoals = func_stats.find('div', attrs={'class':'content-box t-1'}).find('div', attrs={'class':'poss-box'})
    if homeGoals:
        stats_dict['homeGoals'] = homeGoals.get_text('/', strip=True).replace('/Probabilidad gana/',' - ').replace('/Goles esperados','')[6::].replace('-','').replace('%','')
    else:
        stats_dict['homeGoals'] = '-'   

    awayGoals = func_stats.find('div', attrs={'class':'content-box t-2'}).find('div', attrs={'class':'poss-box'})
    if awayGoals:
        stats_dict['awayGoals'] = awayGoals.get_text('/', strip=True).replace('/Probabilidad gana/',' - ').replace('/Goles esperados','')[6::].replace('-','').replace('%','')
    else:
        stats_dict['awayGoals'] = '-'   
        
    awayVictory = func_stats.find('div', attrs={'class':'content-box t-2'}).find('div', attrs={'class':'poss-box'})
    if awayVictory:
        stats_dict['awayVictory'] = awayVictory.get_text('/', strip=True).replace('/Probabilidad gana/',' - ')[0:2].replace('.','').replace('%','')
    else:
        stats_dict['awayVictory'] = '-'     

    awayTeam = func_stats.find('p', attrs={'itemprop':'awayTeam'})
    if awayTeam:
        stats_dict['awayTeam'] = awayTeam.get_text('/', strip=True)
    else:
        stats_dict['awayTeam'] = '-'     
     
    awayRank = func_stats.find('div', attrs={'class':'position t2'})
    if awayRank:
        stats_dict['awayRank'] = awayRank.get_text()
    else:
        stats_dict['awayRank'] = '-' 
        
    competition = func_stats.find('h3', attrs={'class':'competition'})
    if competition:
        stats_dict['competition'] = competition.get_text('/', strip=True)
    else:
        stats_dict['competition'] = '-'
   
    return stats_dict

In [3]:
df = pd.DataFrame()

In [4]:
lista_competiciones = [
    'https://es.besoccer.com/competicion/ligue_1',
    'https://es.besoccer.com/competicion/escocia',
    'https://es.besoccer.com/competicion/portugal',
    'https://es.besoccer.com/competicion/segunda_brasilena',
    'https://es.besoccer.com/competicion/japon',
    'https://es.besoccer.com/competicion/liga_republica_irlanda',
    'https://es.besoccer.com/competicion/premier',
    'https://es.besoccer.com/competicion/segunda',
    'https://es.besoccer.com/competicion/bundesliga_2',
    'https://es.besoccer.com/competicion/j_two_league',
    'https://es.besoccer.com/competicion/brasil',
    'https://es.besoccer.com/competicion/league_one',
    'https://es.besoccer.com/competicion/ligue_2',
    'https://es.besoccer.com/competicion/primera_division_argentina',
    'https://es.besoccer.com/competicion/first_division',
    'https://es.besoccer.com/competicion/vitalis',
    'https://es.besoccer.com/competicion/serie_b',
    'https://es.besoccer.com/competicion/suecia',
    'https://es.besoccer.com/competicion/polonia',
    'https://es.besoccer.com/competicion/resultados/1_league_polonia/2023',
    'https://es.besoccer.com/competicion/resultados/2_league_polonia/2023',
    'https://es.besoccer.com/competicion/resultados/3_league_polonia/2023/grupo1',
    'https://es.besoccer.com/competicion/resultados/3_league_polonia/2023/grupo2',
    'https://es.besoccer.com/competicion/resultados/3_league_polonia/2023/grupo3',
    'https://es.besoccer.com/competicion/resultados/3_league_polonia/2023/grupo4',
    'https://es.besoccer.com/competicion/corea',
    'https://es.besoccer.com/competicion/k_league_corea',
    'https://es.besoccer.com/competicion/national_league_corea',
    'https://es.besoccer.com/competicion/apertura_mexico',
    'https://es.besoccer.com/competicion/liga_de_expansion_mx_apertura',
    'https://es.besoccer.com/competicion/usa'
    'https://es.besoccer.com/competicion/japon',
    'https://es.besoccer.com/competicion/j_two_league',
    'https://es.besoccer.com/competicion/japon_league_3'
    ]

In [5]:
hoy = date.today()
manana = (hoy + timedelta(days=1)).strftime('%Y-%m-%d')
pasado = (hoy + timedelta(days=2)).strftime('%Y-%m-%d')
elotro = (hoy + timedelta(days=3)).strftime('%Y-%m-%d')

lista_diaria = []
url_diaria1 = f'https://es.besoccer.com/livescore/{manana}'
for i in range(1,5):
    url = url_diaria1+f'/{i}'
    lista_diaria.append(url)
url_diaria2 = f'https://es.besoccer.com/livescore/{pasado}'
for i in range(1,5):
    url = url_diaria2+f'/{i}'
    lista_diaria.append(url)
url_diaria3 = f'https://es.besoccer.com/livescore/{elotro}'
for i in range(1,5):
    url = url_diaria3+f'/{i}'
    lista_diaria.append(url)

lista_diaria

['https://es.besoccer.com/livescore/2022-10-13/1',
 'https://es.besoccer.com/livescore/2022-10-13/2',
 'https://es.besoccer.com/livescore/2022-10-13/3',
 'https://es.besoccer.com/livescore/2022-10-13/4',
 'https://es.besoccer.com/livescore/2022-10-14/1',
 'https://es.besoccer.com/livescore/2022-10-14/2',
 'https://es.besoccer.com/livescore/2022-10-14/3',
 'https://es.besoccer.com/livescore/2022-10-14/4',
 'https://es.besoccer.com/livescore/2022-10-15/1',
 'https://es.besoccer.com/livescore/2022-10-15/2',
 'https://es.besoccer.com/livescore/2022-10-15/3',
 'https://es.besoccer.com/livescore/2022-10-15/4']

In [6]:
lista_scrapear = lista_competiciones + lista_diaria
lista_scrapear

['https://es.besoccer.com/livescore/2022-10-13/1',
 'https://es.besoccer.com/livescore/2022-10-13/2',
 'https://es.besoccer.com/livescore/2022-10-13/3',
 'https://es.besoccer.com/livescore/2022-10-13/4',
 'https://es.besoccer.com/livescore/2022-10-14/1',
 'https://es.besoccer.com/livescore/2022-10-14/2',
 'https://es.besoccer.com/livescore/2022-10-14/3',
 'https://es.besoccer.com/livescore/2022-10-14/4',
 'https://es.besoccer.com/livescore/2022-10-15/1',
 'https://es.besoccer.com/livescore/2022-10-15/2',
 'https://es.besoccer.com/livescore/2022-10-15/3',
 'https://es.besoccer.com/livescore/2022-10-15/4',
 'https://es.besoccer.com/competicion/ligue_1',
 'https://es.besoccer.com/competicion/escocia',
 'https://es.besoccer.com/competicion/portugal',
 'https://es.besoccer.com/competicion/segunda_brasilena',
 'https://es.besoccer.com/competicion/japon',
 'https://es.besoccer.com/competicion/liga_republica_irlanda',
 'https://es.besoccer.com/competicion/premier',
 'https://es.besoccer.com/co

In [7]:
for url in lista_scrapear:
    besoccer = requests.get(url)
    s_besoccer = BeautifulSoup(besoccer.text, 'lxml')
    try:
        matches = s_besoccer.find('div', attrs={'class':'comp-matches'}).find_all('a', attrs={'data-status':'-1'})
    except Exception as e1:
        print(i,'.- La url ',url,' no se pudo scrapear')
        print(i,'.-',e1)
        print(i,'.- Probando otra configuración...')
        try:
            matches = s_besoccer.find('div', attrs={'class':'matches'}).find_all('a', attrs={'data-status':'-1'})
        except Exception as e2:
            print(i,'.- La url ',url,' no se pudo scrapear')
            print(i,'.-',e2)

    links_matches = [match.get('href') for match in matches]
    for i in range(len(links_matches)):
        list_analisis = []
        r_match = requests.get(links_matches[i])
        s_match = BeautifulSoup(r_match.text, 'lxml')
        try:
            find_analisis = s_match.find('div', attrs={'class':'menu-scroll'}).select('a[href*="analisis"]')
            analisis = find_analisis[0]
            link_analisis = analisis.get('href')
            r_link_analisis = requests.get(link_analisis)
            s_link_analisis = BeautifulSoup(r_link_analisis.text, 'lxml')
            list_iter = [obtener_stats(s_link_analisis)]
            df_iter = pd.DataFrame(list_iter)
            df = pd.concat([df, df_iter], axis = 0, ignore_index = True)
            print(i,'.- Scrapeando',url)
            print(i,'.-', link_analisis)
        except Exception as e3:
            print(i,'.- La url ',links_matches[i],' no tiene sección análisis')
            print(i,'.-',e3)
            


4 .- La url  https://es.besoccer.com/livescore/2022-10-13/1  no se pudo scrapear
4 .- 'NoneType' object has no attribute 'find_all'
4 .- Probando otra configuración...
0 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/1
0 .- https://es.besoccer.com/partido/fiorentina/heart-of-midlo/2023365827/analisis
1 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/1
1 .- https://es.besoccer.com/partido/austria-wien/villarreal/2023365817/analisis
2 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/1
2 .- https://es.besoccer.com/partido/djurgardens-if/kaa-gent/2023365879/analisis
3 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/1
3 .- https://es.besoccer.com/partido/apollon/az/2023365857/analisis
4 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/1
4 .- https://es.besoccer.com/partido/cfr-cluj/slavia-praha/2023365874/analisis
5 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/1
5 .- https://es.besoccer.com/partido/slovan-bratislava/basel/2

6 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/2
6 .- https://es.besoccer.com/partido/blooming/royal-pari-fc/2022625396/analisis
7 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/2
7 .- https://es.besoccer.com/partido/oriente-petrolero/independiente-petrolero/2022625397/analisis
8 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/2
8 .- https://es.besoccer.com/partido/tampa-bay-rowdies/el-paso-locomotive/2022340925/analisis
9 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/2
9 .- https://es.besoccer.com/partido/birmingham-legion/indy-eleven/2022340954/analisis
10 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/2
10 .- https://es.besoccer.com/partido/tulsa-roughnecks/monterrey-bay-fc/2022340946/analisis
11 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/2
11 .- https://es.besoccer.com/partido/new-mexico-united/la-galaxy-ii/2022340967/analisis
12 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/2
12 .- https://e

13 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/3
13 .- https://es.besoccer.com/partido/al-zulfi/al-ansar-medina/2023486543/analisis
14 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/3
14 .- https://es.besoccer.com/partido/bisha-fc/al-rayyan-fc/2023486544/analisis
15 .- La url  https://es.besoccer.com/partido/weg-taif-sc/al-shuaib-club-saudi/2023486545  no tiene sección análisis
15 .- 'NoneType' object has no attribute 'find'
16 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/3
16 .- https://es.besoccer.com/partido/al-suqoor/al-nairyah-fc/2023486546/analisis
17 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/3
17 .- https://es.besoccer.com/partido/al-jeel/al-safa/2023486547/analisis
18 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/3
18 .- https://es.besoccer.com/partido/mogadishu-city/banadir/2022613377/analisis
19 .- Scrapeando https://es.besoccer.com/livescore/2022-10-13/3
19 .- https://es.besoccer.com/partido/rwamagana-cit

33 .- Scrapeando https://es.besoccer.com/livescore/2022-10-14/1
33 .- https://es.besoccer.com/partido/hermannstadt/fc-rapid-bucuresti/202388150/analisis
34 .- Scrapeando https://es.besoccer.com/livescore/2022-10-14/1
34 .- https://es.besoccer.com/partido/operario-pr/brusque/2022382536/analisis
35 .- Scrapeando https://es.besoccer.com/livescore/2022-10-14/1
35 .- https://es.besoccer.com/partido/ashanti-gb/far-rabat/2023454744/analisis
36 .- Scrapeando https://es.besoccer.com/livescore/2022-10-14/1
36 .- https://es.besoccer.com/partido/fk-arsenal-dzyarzhynsk/minsk/2022398890/analisis
37 .- Scrapeando https://es.besoccer.com/livescore/2022-10-14/1
37 .- https://es.besoccer.com/partido/dinamo-minsk/torpedo-zhodino/2022398884/analisis
38 .- Scrapeando https://es.besoccer.com/livescore/2022-10-14/1
38 .- https://es.besoccer.com/partido/vitebsk/shakhtyor/2022398883/analisis
39 .- Scrapeando https://es.besoccer.com/livescore/2022-10-14/1
39 .- https://es.besoccer.com/partido/slavia/zorka-bdu/2

42 .- Scrapeando https://es.besoccer.com/livescore/2022-10-14/2
42 .- https://es.besoccer.com/partido/waasland-beveren/deinze/202343497/analisis
43 .- Scrapeando https://es.besoccer.com/livescore/2022-10-14/2
43 .- https://es.besoccer.com/partido/panaderia-pulido/ud-tamaraceite/2023294418/analisis
44 .- Scrapeando https://es.besoccer.com/livescore/2022-10-14/2
44 .- https://es.besoccer.com/partido/csd-independiente-valle/mushuc-runa/2022527141/analisis
45 .- Scrapeando https://es.besoccer.com/livescore/2022-10-14/2
45 .- https://es.besoccer.com/partido/celje/tabor-sezana/202343645/analisis
46 .- Scrapeando https://es.besoccer.com/livescore/2022-10-14/2
46 .- https://es.besoccer.com/partido/dom-ale/ask-bravo/202343646/analisis
47 .- Scrapeando https://es.besoccer.com/livescore/2022-10-14/2
47 .- https://es.besoccer.com/partido/yverdon/schaffhausen/202334896/analisis
48 .- Scrapeando https://es.besoccer.com/livescore/2022-10-14/2
48 .- https://es.besoccer.com/partido/lausanne-sports/neuc

...

19 .- Scrapeando https://es.besoccer.com/livescore/2022-10-15/4
19 .- https://es.besoccer.com/partido/zesco-united/real-kings-fc/2023454734/analisis
20 .- Scrapeando https://es.besoccer.com/livescore/2022-10-15/4
20 .- https://es.besoccer.com/partido/milano-united-fc/elgeco-plus/2023454742/analisis
21 .- Scrapeando https://es.besoccer.com/livescore/2022-10-15/4
21 .- https://es.besoccer.com/partido/cs-sfaxien/fasil-ketema-fc/2023454726/analisis
22 .- Scrapeando https://es.besoccer.com/livescore/2022-10-15/4
22 .- https://es.besoccer.com/partido/sagrada-esperanca-327/saint-eloi-lupopo/2023454736/analisis
23 .- Scrapeando https://es.besoccer.com/livescore/2022-10-15/4
23 .- https://es.besoccer.com/partido/asyouty-sport/al-hilal-port-sudan/2023454724/analisis
24 .- Scrapeando https://es.besoccer.com/livescore/2022-10-15/4
24 .- https://es.besoccer.com/partido/rsb-berkane/kwara-united/2023454718/analisis
25 .- Scrapeando https://es.besoccer.com/livescore/2022-10-15/4
25 .- https://es.besoc

4 .- Scrapeando https://es.besoccer.com/competicion/bundesliga_2
4 .- https://es.besoccer.com/partido/magdeburg/eintracht-brau-hellip/202324473/analisis
5 .- Scrapeando https://es.besoccer.com/competicion/bundesliga_2
5 .- https://es.besoccer.com/partido/hannover-96/arminia-bielefeld/202324471/analisis
6 .- Scrapeando https://es.besoccer.com/competicion/bundesliga_2
6 .- https://es.besoccer.com/partido/paderborn/sv-sandhausen/202324468/analisis
7 .- Scrapeando https://es.besoccer.com/competicion/bundesliga_2
7 .- https://es.besoccer.com/partido/holstein-kiel/heidenheim/202324469/analisis
8 .- Scrapeando https://es.besoccer.com/competicion/bundesliga_2
8 .- https://es.besoccer.com/partido/kaiserslautern/ssv-jahn-regensburg/202324474/analisis
0 .- Scrapeando https://es.besoccer.com/competicion/j_two_league
0 .- https://es.besoccer.com/partido/tokyo-verdy/albirex-niigata/2022356118/analisis
1 .- Scrapeando https://es.besoccer.com/competicion/j_two_league
1 .- https://es.besoccer.com/parti

2 .- Scrapeando https://es.besoccer.com/competicion/first_division
2 .- https://es.besoccer.com/partido/ayr-united/partick-thistle-fc/202323323/analisis
0 .- Scrapeando https://es.besoccer.com/competicion/vitalis
0 .- https://es.besoccer.com/partido/cf-estrela-de-amadora/sporting-covilha/202396363/analisis
1 .- Scrapeando https://es.besoccer.com/competicion/vitalis
1 .- https://es.besoccer.com/partido/benfica-ii/mafra/202396357/analisis
2 .- Scrapeando https://es.besoccer.com/competicion/vitalis
2 .- https://es.besoccer.com/partido/torreense/cf-os-belenenses-ii/202396360/analisis
3 .- Scrapeando https://es.besoccer.com/competicion/vitalis
3 .- https://es.besoccer.com/partido/feirense/moreirense-fc/202396356/analisis
4 .- Scrapeando https://es.besoccer.com/competicion/vitalis
4 .- https://es.besoccer.com/partido/porto-ii/penafiel/202396361/analisis
5 .- Scrapeando https://es.besoccer.com/competicion/vitalis
5 .- https://es.besoccer.com/partido/academico-viseu/leixoes/202396362/analisis


0 .- Scrapeando https://es.besoccer.com/competicion/resultados/3_league_polonia/2023/grupo1
0 .- https://es.besoccer.com/partido/legia-warszawa-ii/olimpia-zambrow/2023276050/analisis
1 .- Scrapeando https://es.besoccer.com/competicion/resultados/3_league_polonia/2023/grupo1
1 .- https://es.besoccer.com/partido/ks-blonianka-blonie/pelikan-lowicz/2023276043/analisis
2 .- Scrapeando https://es.besoccer.com/competicion/resultados/3_league_polonia/2023/grupo1
2 .- https://es.besoccer.com/partido/sokol-ostroda/jagiellonia-ii/2023276045/analisis
3 .- Scrapeando https://es.besoccer.com/competicion/resultados/3_league_polonia/2023/grupo1
3 .- https://es.besoccer.com/partido/swit-nowy-dwor/lks-lodz-ii/2023276048/analisis
4 .- Scrapeando https://es.besoccer.com/competicion/resultados/3_league_polonia/2023/grupo1
4 .- https://es.besoccer.com/partido/mks-mlawianka-mlawa/concordia-elblag/2023276049/analisis
5 .- Scrapeando https://es.besoccer.com/competicion/resultados/3_league_polonia/2023/grupo1
5

6 .- Scrapeando https://es.besoccer.com/competicion/national_league_corea
6 .- https://es.besoccer.com/partido/siheung-citizen/changwon-city-kr/2022452441/analisis
7 .- Scrapeando https://es.besoccer.com/competicion/national_league_corea
7 .- https://es.besoccer.com/partido/gyeongju-hnp/dangjin-citizen-fc/2022452442/analisis
0 .- Scrapeando https://es.besoccer.com/competicion/liga_de_expansion_mx_apertura
0 .- https://es.besoccer.com/partido/club-atletico-la-paz/cd-tapatio/202321049/analisis
1 .- Scrapeando https://es.besoccer.com/competicion/liga_de_expansion_mx_apertura
1 .- https://es.besoccer.com/partido/cimarrones-de-sonora/merida/202321050/analisis
2 .- Scrapeando https://es.besoccer.com/competicion/liga_de_expansion_mx_apertura
2 .- https://es.besoccer.com/partido/morelia/universidad-gu/202321052/analisis
3 .- Scrapeando https://es.besoccer.com/competicion/liga_de_expansion_mx_apertura
3 .- https://es.besoccer.com/partido/durango/mineros-de-zacatecas/202321053/analisis
4 .- Scra

In [8]:
# cuenta el número de registros encontrados

print(f'Número de registros scrapeados: {len(df)}')

Número de registros scrapeados: 698


In [9]:
# busca si hay registros duplicados

df.duplicated()

# elimina los registros duplicados

df = df.drop_duplicates()

In [10]:
# cuenta el número de registros restantes después de eliminar duplicados

print(f'Número de registros después de haber eliminado los duplicados: {len(df)}')

Número de registros después de haber eliminado los duplicados: 612


In [11]:
df['draw'] = 100 - df['homeVictory'].astype(int) - df['awayVictory'].astype(int)

In [44]:
# df['zerozero'].astype('float64')
# type(df['zerozero'])

pandas.core.series.Series

In [12]:
# for match in df.iterrows():
#     index_value, column_values=match
#     arreglo=float(column_values['zerozero'].replace('<',''))
#     df.at[index_value,'zerozero']=arreglo

In [13]:
print('Partidos con poco porcentaje de acierto')
print('- ' * 10)
print(f'Número de registros actuales: {len(df)}')
print('- ' * 10)
print('- ' * 10)
print('- ' * 10)
# for item in df.index:
#     a = df['zerozero'][item]
#     if a < 7:
#         df.drop(df['zerozero'], inplace=True)
#     print(a)

                   
                   
    #if a < 7:
        #print(a)
#     if a > 0:
#         df['zerozero'] = df['zerozero'].astype('float64')
#     else:
#         pass 
    
#     for item in df.index:
#     a = df.homeTeam[item]
#     if a in df_reemplazar.values:
#         posicionHome = df_reemplazar[df_reemplazar['originalTeam'] == a].index.tolist()
#         for e in posicionHome:
#             newTeam = df_reemplazar.betTeam[e]
#             df = df.replace({a}, {newTeam}, regex=False)
#             print(f'Equipo renombrado: {a} --> {newTeam}')
#     else:
#         pass 



# type(df['a'][1])
# Out[25]: str

# float(df['a'][1])
# Out[26]: 0.123

# type(float(df['a'][1]))
# Out[27]: float

Partidos con poco porcentaje de acierto
- - - - - - - - - - 
Número de registros actuales: 612
- - - - - - - - - - 
- - - - - - - - - - 
- - - - - - - - - - 


In [30]:
hora = '%H'
ahora = datetime.now().strftime(hora)
besoccerzone = timezone('Etc/Greenwich')
besoccertime = datetime.now(besoccerzone).strftime(hora)
diferencia_horaria = int(ahora) - int(besoccertime)
df['date'] = pd.to_datetime(df["date"]) + pd.Timedelta(hours=diferencia_horaria)

In [31]:
df = df.reindex(columns=['date','zerozero','homeRank','homeTeam','homeGoals','homeVictory','draw','awayVictory','awayGoals','awayTeam','awayRank','competition'])   

In [32]:
df = df.round(decimals = 3)

In [33]:
dict_reemplazar = {
    'originalTeam': [
        'Alavés',
        'América de Cali',
        'Atl.Tucumán',
        'Atlético',
        'CD Águila',
        'CD Lugo',
        'CD Olmedo',
        'Cobán Imperial',
        'Colón',
        'Djurgårdens IF',
        'FC St. Pauli',
        'Ferencvárosi',
        'Gloria Buzău',
        'Guaraní',
        'IFK Malmö',
        'İstanbul Başakşehir',
        'Jönköpings Södra',
        'Lanús',
        'Leganés',
        'LKS Lódz',
        'Mirandés',
        'Nacional Potosí',
        'Patriotas Boyacá',
        'Qarabağ',
        'Raya2 Expansión',
        'Real Betis',
        'Águilas Doradas',
        'Deportes Tolima',
        'HB Chelghoum Laid',
        'At. Nacional',
        'Colón'
    ],
    'betTeam': [
        'Alaves',
        'America de Cali',
        'Atl Tucuman',
        'Atletico',
        'CD Aguila',
        'Lugo',
        'Olmedo',
        'Coban Imperial',
        'Colon',
        'Djurgardens IF',
        'St Pauli',
        'Ferencvarosi',
        'Gloria Buzau',
        'Guarani',
        'Malmo IF',
        'Basaksehir',
        'Jonkopings Sodra',
        'Lanus',
        'Leganes',
        'LKS Lodz',
        'Mirandes',
        'Nacional Potosi',
        'Boyaca Patriotas',
        'Qarabag',
        'Raya2',
        'Betis',
        'Aguilas Doradas Rionegro',
        'Tolima',
        'Chelghoum Laid',
        'Atletico Nacional Medellin',
        'Colon'
    ]
}
df_reemplazar = pd.DataFrame(dict_reemplazar)
#df_reemplazar.sort_values(['betTeam'])

In [46]:
# cambia nombre de equipos en columna home
print('Renombrando equipos de columna home')
print('- ' * 10) 
for item in df.index:
    a = df.homeTeam[item]
    if a in df_reemplazar.values:
        posicionHome = df_reemplazar[df_reemplazar['originalTeam'] == a].index.tolist()
        for e in posicionHome:
            newTeam = df_reemplazar.betTeam[e]
            df = df.replace({a}, {newTeam}, regex=False)
            print(f'Equipo renombrado: {a} --> {newTeam}')
    else:
        pass 

Renombrando equipos de columna home
- - - - - - - - - - 


In [47]:
# cambia nombre de equipos en columna away
print('Renombrando equipos de columna away')
print('- ' * 10) 
for item in df.index:
    a = df.awayTeam[item]
    if a in df_reemplazar.values:
        posicionAway = df_reemplazar[df_reemplazar['originalTeam'] == a].index.tolist()
        for e in posicionAway:
            newTeam = df_reemplazar.betTeam[e]
            df = df.replace({a}, {newTeam}, regex=False)
            print(f'Equipo renombrado: {a} --> {newTeam}')
    else:
        pass

Renombrando equipos de columna away
- - - - - - - - - - 


In [36]:
dict_eliminar = {
    'borrarLiga': [
        'Rusia', 
        'Uefa Youth League',
        'CONCACAF',
        'Segunda Sudáfrica',
        'Segunda Indonesia',
        'Tercera Georgia',
        'Liga Somalia',
        'Liga Ruanda'
    ]
}

df_borrarLiga = pd.DataFrame(dict_eliminar)

In [48]:
# elimina equipos de ligas no requeridas
print('Eliminar partidos de ligas no deseadas')
print('- ' * 10) 
print(f'Número de registros actualmente: {len(df)}')
print('- ' * 10)        
for item in df_borrarLiga.index:
    a = df_borrarLiga.borrarLiga[item]
    df.drop(df[df['competition'].str.contains(a)].index, inplace=True)
    print(f'Liga {a}')
else:
    pass
    #print(f'No hay coindicidencias en: {a}')
        
        
        
print('- ' * 10)        
print(f'Número de registros después de eliminar las ligas no deseadas: {len(df)}')
print(' -' * 10)    

Eliminar partidos de ligas no deseadas
- - - - - - - - - - 
Número de registros actualmente: 595
- - - - - - - - - - 
Liga Rusia
Liga Uefa Youth League
Liga CONCACAF
Liga Segunda Sudáfrica
Liga Segunda Indonesia
Liga Tercera Georgia
Liga Liga Somalia
Liga Liga Ruanda
- - - - - - - - - - 
Número de registros después de eliminar las ligas no deseadas: 595
 - - - - - - - - - -


In [39]:
df.style.set_properties(**{'text-align': 'center'})

,date,zerozero,homeRank,homeTeam,homeGoals,homeVictory,draw,awayVictory,awayGoals,awayTeam,awayRank,competition
0,2022-10-13 22:45:00,2.8,2º,Fiorentina,2.45,66,19,15,1.12,Heart of Midlothian,3º,"Conference League. Grupo A, Jornada 4"
1,2022-10-13 22:45:00,6.1,4º,Austria Wien,0.86,16,22,62,1.93,Villarreal,1º,"Conference League. Grupo C, Jornada 4"
2,2022-10-13 22:45:00,4.5,1º,Djurgardens IF,1.7,44,25,31,1.41,KAA Gent,2º,"Conference League. Grupo F, Jornada 4"
3,2022-10-13 22:45:00,2.5,4º,Apollon Limassol,.88,9,14,77,2.82,AZ Alkmaar,1º,"Conference League. Grupo E, Jornada 4"
4,2022-10-13 22:45:00,5.5,1º,CFR Cluj,1.4,35,26,39,1.5,Slavia Praha,2º,"Conference League. Grupo G, Jornada 4"
5,2022-10-13 22:45:00,4.9,3º,Slovan Bratislava,1.47,36,25,39,1.54,Basel,2º,"Conference League. Grupo H, Jornada 4"
6,2022-10-13 22:45:00,1.5,1º,Partizan Belgrade,1.67,24,20,56,2.54,Köln,3º,"Conference League. Grupo D, Jornada 4"
7,2022-10-13 22:45:00,2.8,1º,Basaksehir,2.25,58,21,21,1.33,FK RFS,4º,"Conference League. Grupo A, Jornada 4"
8,2022-10-14 01:00:00,8.3,1º,West Ham,1.37,42,28,30,1.12,Anderlecht,2º,"Conference League. Grupo B, Jornada 4"
9,2022-10-14 01:00:00,5.5,2º,Nice,1.81,54,24,22,1.09,Slovácko,4º,"Conference League. Grupo D, Jornada 4"


In [38]:
# ordena un DataFrame por columna
# se puede ordenar por varias columnas añadiendo el nombre de la siguiente columna e.j. ('date', 'competition')

df.sort_values(['date', 'competition']).to_csv('matches.csv', header = True)

In [24]:
################################################################################################
################################################################################################
## REVISIÓN DE VERSIONES                                                                      ##
##                                                                                            ##
## v.3.2.                                                                                     ##
##  - añadido filtro para cambiar nombres de equipos                                          ##
##  - añadido comando para eliminar ciertos registros segúna las ligas que no interesan       ##
##                                                                                            ##
## v.3.1.                                                                                     ##
## - Menú para elegir si scrapear sólo competiciones, sólo partidos diarios o todo (VS Code)  ##
## - limpieza de código                                                                       ##
## - sustituido .append por pd.concat                                                         ##
## - detecta automáticamente los tres días siguientes para escrapear partidos diarios         ##
## - detecta automáticamente zona horaria para ajustar hora de los partidos                   ##
##                                                                                            ##
## v.3.0                                                                                      ##
## - hace scraping de uns lista de competiciones y página de partidos diarios                 ##
## - busca registros duplicados y los elimina                                                 ##
## - añade columna con los porcentajes de partidos NoGoal                                     ##
##                                                                                            ##
## v.2.1                                                                                      ##
## - añade la paginación de los partidos diarios                                              ##
##                                                                                            ##
## v.2.0                                                                                      ##
## - scraping de una página específica de partidos diarios                                    ##
## - crea dataframe y lo envía a archivo .csv                                                 ##
##                                                                                            ##
################################################################################################
################################################################################################

In [25]:
# se reemplaza el nombre de algunos equipos para ser compatibles con las plataformas
# df = df.replace("Credicoop San Cristóbal", "Credicoop San Cristobal")

In [26]:
# se eliminan registros que incluyan competiciones que no interesan
# df.drop(df[df['competition'].str.contains('Apertura')].index, inplace=True)

In [27]:
######################################################################
## métodos para trabajar con los datos de pandas                    ##
##                                                                  ##
## del(df['newcolumn'])  --- elimina una columna                    ##
## df.rename(columns = {'NoGoal':'NoGoal%'}, inplace = True)        ##
######################################################################

In [28]:
# df.loc[df.competition.str.contains('liga escocia', case = False), "NoGoal%"] = '92.85'
# df.loc[df.competition.str.contains('serie b - brasil', case = False), "NoGoal%"] = '92.06'
# df.loc[df.competition.str.contains('liga fútbol japón', case = False), "NoGoal%"] = '88.67'
# df.loc[df.competition.str.contains('liga irlanda.', case = False), "NoGoal%"] = '87.67'
# df.loc[df.competition.str.contains('segunda división.', case = False), "NoGoal%"] = '86.36'
# df.loc[df.competition.str.contains('segunda japonesa j2', case = False), "NoGoal%"] = '84.72'
# df.loc[df.competition.str.contains('league one.', case = False), "NoGoal%"] = '84.25'
# df.loc[df.competition.str.contains('liga profesional argentina', case = False), "NoGoal%"] = '83.45'
# df.loc[df.competition.str.contains('ligue 2', case = False), "NoGoal%"] = '83.14'